<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [3]:
# Install category_encoders

!pip install category_encoders

     |████████████████████████████████| 81kB 3.6MB/s 


In [4]:
# Install keras-tuner

!pip install keras-tuner

     |████████████████████████████████| 61kB 3.8MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=7f50b624092d302d8cd2af6573d1bfc8bd938ab47252f5e8b1855d6d9342a304
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=752164d7f3e2d500a4685a0c9bf5d561a7dc7ac9086b06ec8eb4a72f04e7ea74
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [5]:
# Imports

import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import os
import datetime
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from kerastuner.tuners import RandomSearch
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
%load_ext tensorboard

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [7]:
# Import the data

df = pd.read_csv('https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')

In [8]:
# See df

df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [9]:
# df shape

df.shape

(7043, 21)

In [10]:
# Check for nans

df.isnull().values.any()

False

In [11]:
# Baseline

df['Churn'].value_counts(normalize=True)

No     0.73463
Yes    0.26537
Name: Churn, dtype: float64

In [12]:
# Convert categries to binary

df['Churn'] = df['Churn'].replace({'Yes':1, 'No':0})

In [13]:
# Check unique values in df

for col in df.columns: print(col, df[col].nunique())

customerID 7043
gender 2
SeniorCitizen 2
Partner 2
Dependents 2
tenure 73
PhoneService 2
MultipleLines 3
InternetService 3
OnlineSecurity 3
OnlineBackup 3
DeviceProtection 3
TechSupport 3
StreamingTV 3
StreamingMovies 3
Contract 3
PaperlessBilling 2
PaymentMethod 4
MonthlyCharges 1585
TotalCharges 6531
Churn 2


In [14]:
# Drop customerID, too much noise and drop target

X = df.drop(columns=['Churn', 'customerID']).values
y = df['Churn'].values

In [15]:
# Check data types

df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                 int64
dtype: object

In [16]:
# Ordinal encode so many low cardinality categories

encode = ce.OrdinalEncoder()

X = encode.fit_transform(X)

In [17]:
# Scale the data for the neural network

#scale = MinMaxScaler()

scale = StandardScaler()

X = scale.fit_transform(X)

In [18]:
# Train Test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=7)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(4930, 19) (2113, 19) (4930,) (2113,)


In [ ]:
# GridSearch and Cross Validation for initial pass

# Fix random seed for reproducibility

seed = 7
np.random.seed(seed)

# Function to create model, required for KerasClassifier

def create_model(lr):

    # Create optimizer

    adam = Adam(learning_rate=lr)

    # Create model

    model = Sequential()
    model.add(Dense(19, input_dim=19, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

# Create model

model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the grid search parameters

param_grid = {'lr': [.001, .01, .1, .2, .3, .5],
              'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [100]}

# Create Grid Search

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

# 1st Best: 0.787221086025238 using {'batch_size': 40, 'epochs': 100, 'lr': 0.001}
# This was run without GPU, just CPU, and it took about 30 minutes

In [ ]:
#%tensorboard --logdir logs

In [ ]:
# RandomSearch and Cross Validation for initial pass

# Fix random seed for reproducibility

seed = 7
np.random.seed(seed)

# Function to create model, required for KerasClassifier

def create_model(lr):

    # Create optimizer

    adam = Adam(learning_rate=lr)

    # Create model

    model = Sequential()
    model.add(Dense(19, input_dim=19, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

# Create model

model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the grid search parameters

param_grid = {'lr': [.001, .01, .1, .2, .3, .5],
              'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [100]}

# Create Grid Search

grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

# 1st Best: 0.786612582206726 using {'lr': 0.1, 'epochs': 100, 'batch_size': 100}
# This was run with GPU and it also took about 30 minutes, 0.0006 worse than Gridsearch

Best: 0.786612582206726 using {'lr': 0.1, 'epochs': 100, 'batch_size': 100}
Means: 0.786612582206726, Stdev: 0.007274001630058567 with: {'lr': 0.1, 'epochs': 100, 'batch_size': 100}
Means: 0.7746450424194335, Stdev: 0.007761119619711015 with: {'lr': 0.1, 'epochs': 100, 'batch_size': 10}
Means: 0.7829614639282226, Stdev: 0.00650985323103509 with: {'lr': 0.001, 'epochs': 100, 'batch_size': 80}
Means: 0.7762677550315857, Stdev: 0.01015416027683366 with: {'lr': 0.01, 'epochs': 100, 'batch_size': 10}
Means: 0.776673436164856, Stdev: 0.0060648663294483734 with: {'lr': 0.001, 'epochs': 100, 'batch_size': 60}
Means: 0.7807302355766297, Stdev: 0.006459097335944383 with: {'lr': 0.01, 'epochs': 100, 'batch_size': 40}
Means: 0.7807302236557007, Stdev: 0.010606021239098977 with: {'lr': 0.2, 'epochs': 100, 'batch_size': 20}
Means: 0.778904664516449, Stdev: 0.00580845114156917 with: {'lr': 0.2, 'epochs': 100, 'batch_size': 40}
Means: 0.7791074991226197, Stdev: 0.006132353571338926 with: {'lr': 0.3, '

In [ ]:
#%tensorboard --logdir logs

In [26]:
# GridSearch and Cross Validation for second pass of the following hyperparameters:

# batch_size
# training epochs
# optimizer
# learning rate (if applicable to optimizer)
# momentum (if applicable to optimizer)
# activation functions
# network weight initialization
# dropout regularization
# number of neurons in the hidden layer

# All of the hyperparameters

#param_grid = {'batch_size' : [10, 20, 40, 60, 80, 100],
#              'epochs' : [50, 75, 100, 125, 150],
#              'optimizer' : ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'],
#              'lr' : [.001, .01, .1],
#              'momentum' : [0.0, 0.2, 0.4, 0.6, 0.8, 0.9],
#              'activation' : ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear'],
#              'net_weight_init' : ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform'],
#              'dropout_rate' : [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
#              'neurons' : [1, 5, 10, 15, 20, 25, 30],
#              'l1_reg': [1e-2, le-3, le-4], # weight decay, add this term to loss function, small coeffecient
#              'l2_reg': [1e-2, le-3, le-4]}

# However, there are too many options, and colab will time-out in 12 hours before
# the model finishes running, so trim this down.


# Fix random seed for reproducibility

seed = 7
np.random.seed(seed)

# Function to create model, required for KerasClassifier

def create_model(batch_size, epochs, optimizer, lr, momentum, activation, 
                 net_weight_init, dropout_rate, neurons, l1_reg, l2_reg):

    # Create optimizer

    adam = Adam(learning_rate=lr)

    # Create model

    model = Sequential()
    model.add(Dense(19, input_dim=19, activation=activation))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Create model

model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the grid search parameters

param_grid = {'batch_size' : [60, 80],
              'epochs' : [100],
              'optimizer' : ['Adam', 'Nadam'],
              'lr' : [.001, .01],
              'momentum' : [0.1, 0.2],
              'activation' : ['relu'],
              'net_weight_init' : ['uniform', 'lecun_uniform'],
              'dropout_rate' : [0.0, 0.1],
              'neurons' : [1, 10],
              'l1_reg': [1e-5],
              'l2_reg': [1e-5]}

# Create Grid Search

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

# 1st Best: 0.7912778854370117 using {'activation': 'relu', 'batch_size': 80, 'dropout_rate': 0.0, 'epochs': 10, 'l1_reg': 1e-05, 'l2_reg': 1e-05, 'lr': 0.01, 'momentum': 0.2, 'net_weight_init': 'lecun_uniform', 'neurons': 1, 'optimizer': 'Adam'}
# This was run with CPU, and it took about 10 minutes
# 2nd Best: Best: 0.7876267790794372 using {'activation': 'relu', 'batch_size': 80, 'dropout_rate': 0.1, 'epochs': 100, 'l1_reg': 1e-05, 'l2_reg': 1e-05, 'lr': 0.01, 'momentum': 0.1, 'net_weight_init': 'lecun_uniform', 'neurons': 10, 'optimizer': 'Nadam'}

Best: 0.7876267790794372 using {'activation': 'relu', 'batch_size': 80, 'dropout_rate': 0.1, 'epochs': 100, 'l1_reg': 1e-05, 'l2_reg': 1e-05, 'lr': 0.01, 'momentum': 0.1, 'net_weight_init': 'lecun_uniform', 'neurons': 10, 'optimizer': 'Nadam'}
Means: 0.7815415740013123, Stdev: 0.006653643495390422 with: {'activation': 'relu', 'batch_size': 60, 'dropout_rate': 0.0, 'epochs': 100, 'l1_reg': 1e-05, 'l2_reg': 1e-05, 'lr': 0.001, 'momentum': 0.1, 'net_weight_init': 'uniform', 'neurons': 1, 'optimizer': 'Adam'}
Means: 0.7791074991226197, Stdev: 0.009522676594026164 with: {'activation': 'relu', 'batch_size': 60, 'dropout_rate': 0.0, 'epochs': 100, 'l1_reg': 1e-05, 'l2_reg': 1e-05, 'lr': 0.001, 'momentum': 0.1, 'net_weight_init': 'uniform', 'neurons': 1, 'optimizer': 'Nadam'}
Means: 0.7874239206314086, Stdev: 0.010234870703621352 with: {'activation': 'relu', 'batch_size': 60, 'dropout_rate': 0.0, 'epochs': 100, 'l1_reg': 1e-05, 'l2_reg': 1e-05, 'lr': 0.001, 'momentum': 0.1, 'net_weight_init': 

In [ ]:
%tensorboard --logdir logs

In [25]:
# GridSearch and Cross Validation for second pass of the following hyperparameters:

# batch_size
# training epochs
# optimizer
# learning rate (if applicable to optimizer)
# momentum (if applicable to optimizer)
# activation functions
# network weight initialization
# dropout regularization
# number of neurons in the hidden layer

# All of the hyperparameters

#param_grid = {'batch_size' : [10, 20, 40, 60, 80, 100],
#              'epochs' : [50, 75, 100, 125, 150],
#              'optimizer' : ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'],
#              'lr' : [.001, .01, .1],
#              'momentum' : [0.0, 0.2, 0.4, 0.6, 0.8, 0.9],
#              'activation' : ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear'],
#              'net_weight_init' : ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform'],
#              'dropout_rate' : [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
#              'neurons' : [1, 5, 10, 15, 20, 25, 30],
#              'l1_reg': [1e-2, le-3, le-4], # weight decay, add this term to loss function, small coeffecient
#              'l2_reg': [1e-2, le-3, le-4]}

# However, there are too many options, and colab will time-out in 12 hours before
# the model finishes running, so trim this down.


# Fix random seed for reproducibility

seed = 7
np.random.seed(seed)

# Function to create model, required for KerasClassifier

def create_model(batch_size, epochs, optimizer, lr, momentum, activation, 
                 net_weight_init, dropout_rate, neurons, l1_reg, l2_reg):

    # Create optimizer

    adam = Adam(learning_rate=lr)

    # Create model

    model = Sequential()
    model.add(Dense(19, input_dim=19, activation=activation))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Create model

model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the grid search parameters

param_grid = {'batch_size' : [80, 100],
              'epochs' : [100],
              'optimizer' : ['Adam', 'Nadam'],
              'lr' : [.01, .1],
              'momentum' : [0.0, 0.2],
              'activation' : ['relu'],
              'net_weight_init' : ['uniform', 'lecun_uniform'],
              'dropout_rate' : [0.0, 0.1],
              'neurons' : [1, 5],
              'l1_reg': [1e-5],
              'l2_reg': [1e-5]}

# Create Grid Search

grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

# 1st Best: 0.7896551728248596 using {'optimizer': 'Nadam', 'neurons': 1, 'net_weight_init': 'uniform', 'momentum': 0.2, 'lr': 0.01, 'l2_reg': 1e-05, 'l1_reg': 1e-05, 'epochs': 10, 'dropout_rate': 0.1, 'batch_size': 80, 'activation': 'relu'}
# This was run with CPU, and it took about 10 minutes
# 2nd Best: Best: 0.7851926922798157 using {'optimizer': 'Adam', 'neurons': 1, 'net_weight_init': 'uniform', 'momentum': 0.2, 'lr': 0.1, 'l2_reg': 1e-05, 'l1_reg': 1e-05, 'epochs': 100, 'dropout_rate': 0.1, 'batch_size': 100, 'activation': 'relu'}

Best: 0.7851926922798157 using {'optimizer': 'Adam', 'neurons': 1, 'net_weight_init': 'uniform', 'momentum': 0.2, 'lr': 0.1, 'l2_reg': 1e-05, 'l1_reg': 1e-05, 'epochs': 100, 'dropout_rate': 0.1, 'batch_size': 100, 'activation': 'relu'}
Means: 0.7821501016616821, Stdev: 0.008519264629910673 with: {'optimizer': 'Adam', 'neurons': 1, 'net_weight_init': 'uniform', 'momentum': 0.2, 'lr': 0.1, 'l2_reg': 1e-05, 'l1_reg': 1e-05, 'epochs': 100, 'dropout_rate': 0.0, 'batch_size': 100, 'activation': 'relu'}
Means: 0.7851926922798157, Stdev: 0.007125412921382508 with: {'optimizer': 'Adam', 'neurons': 1, 'net_weight_init': 'uniform', 'momentum': 0.2, 'lr': 0.1, 'l2_reg': 1e-05, 'l1_reg': 1e-05, 'epochs': 100, 'dropout_rate': 0.1, 'batch_size': 100, 'activation': 'relu'}
Means: 0.7803245425224304, Stdev: 0.00473967400478512 with: {'optimizer': 'Nadam', 'neurons': 5, 'net_weight_init': 'lecun_uniform', 'momentum': 0.2, 'lr': 0.1, 'l2_reg': 1e-05, 'l1_reg': 1e-05, 'epochs': 100, 'dropout_rate': 0.1, '

In [ ]:
%tensorboard --logdir logs

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?